In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
    .master("spark://de-zoomcamp.us-west4-b.c.projectonleali.internal:7077")\
        .appName('TestMaster01')\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/23 22:43:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
mi_parquet = spark.read.parquet("../data/data/green/*/*")

In [5]:
mi_parquet.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [6]:
mi_parquet.createOrReplaceTempView('mi_vista')

In [7]:
mi_totalizador = spark.sql(
    """
    select PULocationID as ZoneID,
    date_trunc('day', lpep_pickup_datetime) as dateID,
    sum(total_amount) as total
    from mi_vista
    group by ZoneID, dateID
    """
)

In [8]:
print(mi_totalizador.printSchema())

root
 |-- ZoneID: long (nullable = true)
 |-- dateID: timestamp (nullable = true)
 |-- total: double (nullable = true)

None


In [9]:
mi_totalizador.coalesce(1).write.parquet("../data/reportes/", mode='overwrite')